<img width="8%" alt="YouTube.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/YouTube.png" style="border-radius: 15%">

# YouTube - Extract Clip and Generate Content Ideas

**Tags:** #content #youtube #generativeai #genai #chat #ai #automation #prompt #command

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/)

**Last update:** 2023-10-10 (Created: 2023-09-22)

**Description:**
This notebook template is a powerful tool for content creators and marketers who want to leverage YouTube videos to extract valuable insights and generate content ideas. With this template, you can easily extract clips from YouTube videos, capture key moments, and brainstorm creative content ideas based on the extracted clips.

**Key Features:**

1️⃣ YouTube Video Extraction: Seamlessly extract YouTube videos using the YouTube Data API, allowing you to access a vast library of content for analysis and inspiration.

2️⃣ Clip Extraction: Effortlessly extract specific clips from the YouTube videos, enabling you to focus on the most relevant and engaging moments that resonate with your target audience.

3️⃣ Content Ideas Generation: Leverage the extracted clips to spark your creativity and generate a wide range of content ideas. Whether you're creating blog posts, social media content, or video scripts, this template will help you brainstorm innovative and captivating ideas.

4️⃣ Timestamp Integration: Automatically integrate timestamps into the extracted clips, making it easy to reference and navigate to specific moments within the original YouTube videos.

5️⃣ Collaboration and Sharing: Collaborate with your team by sharing the notebook template, allowing everyone to contribute their ideas and insights. Foster a creative environment where the power of collective thinking fuels your content creation process.

6️⃣ Documentation and Export: Document your content ideas, insights, and analysis within the notebook template. Export your findings to various formats, including PDF, Markdown, or HTML, for easy sharing and reference.

Whether you're a content creator, marketer, or strategist, this notebook template empowers you to extract, analyze, and generate captivating content ideas from YouTube videos. Unleash your creativity, tap into the vast YouTube library, and create content that resonates with your audience like never before. 🚀💡 #YouTubeContentIdeas #ContentCreation #VideoMarketing

## Input

### Import libraries

In [ ]:
import naas
try:
    from pytube import YouTube
except:
    !pip install pytube --user
    from pytube import YouTube
try:
    import moviepy
except:
    !pip install moviepy --user
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import AudioFileClip, VideoFileClip
try:
    from tqdm import tqdm
except:
    !pip install tqdm --user
    from tqdm import tqdm
import openai
import concurrent.futures
import math
import os
import naas_data_product

### Setup variables

In [ ]:
# Mandatory
youtube_url = "https://www.youtube.com/watch?v=-DVyjdw4t9I"
start_time = "00:00:00"
end_time = "00:06:00"
name = "Guido van Rossum: Code Readbilty, Indentations and Bugs"

# Optional
openai.api_key = naas.secret.get("OPENAI_API_KEY")
text_model = "gpt-4"
audio_model = "whisper-1"
output_dir = os.path.join(naas_data_product.ROOT_PATH, "outputs", "content-engine", "YouTube_Extract_Clip_and_Generate_Content_Ideas")
os.makedirs(output_dir, exist_ok=True) # Create dirs
video_file_path = os.path.join(output_dir,  f"{name}.3gpp")
clip_file_path = os.path.join(output_dir,  f"{name}.mp4")
audio_file_path = clip_file_path.replace('.mp4', '.mp3')
content_file_path = os.path.join(output_dir,  f"Content_{name}.md")

# Webhook body
body = {}

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [ ]:
# Parameters
if len(body) > 0:
    youtube_url = body.get("youtube_url")
    start_time = body.get("start_time")
    end_time = body.get("end_time")
    name = body.get("name")

## Model

### Extract Video Clip

In [ ]:
def convert_to_seconds(end):
    h, m, s = map(int, end.split(':'))
    return h * 3600 + m * 60 + s

def extract_video_clip(
    youtube_url,
    video_file_path,
    clip_file_path,
    start_time,
    end_time,
):
    # Download the video
    yt = YouTube(f"{youtube_url}")

    # Check if there are any streams available
    if yt.streams:
        stream = yt.streams.first()

        video_filename = stream.download(output_path=video_file_path)

        # Start and end times in seconds
        start_time = convert_to_seconds(start_time)
        end_time = convert_to_seconds(end_time)

        # Cut the video
        ffmpeg_extract_subclip(video_filename, start_time, end_time, targetname=clip_file_path)
    else:
        print("No streams available for this video.")
        
extract_video_clip(
    youtube_url,
    video_file_path,
    clip_file_path,
    start_time,
    end_time
)

### Transforming video into audio

In [ ]:
def transform_video_to_audio(clip_file_path, audio_file_path):
    videoclip = VideoFileClip(clip_file_path)
    audioclip = videoclip.audio
    audioclip.write_audiofile(audio_file_path)
    
transform_video_to_audio(
    clip_file_path,
    audio_file_path
)

### Split audio files in smaller chunks

This is actually needed to not reach limits owhen transribing audio

In [ ]:
def chunk_audio_file(
    output_dir,
    audio_file_path,
    chunk_length=300
):
    # Storing chunks that will be returned.
    chunks = [] 
    
    # Load audio clip file.
    audioclip = AudioFileClip(audio_file_path)

    # Determine the duration of the audio file in seconds
    duration = audioclip.duration

    # Calculate the number of chunks
    num_chunks = int(math.ceil(duration / chunk_length))

    # Split the audio file and write each chunk to a new file
    for i in range(num_chunks):
        start_time = i * chunk_length
        end_time = (i + 1) * chunk_length
        
        # Make sure that if the end_time is superior to duration we set it to duration.
        # This will happen on the last chunk most of the time.
        if end_time > duration:
            end_time = duration
            
        chunk = audioclip.subclip(start_time, end_time)
        audio_chunk_path = os.path.join(output_dir, f"chunk_{i}.mp3")
        
        # Write audio chunk
        chunk.write_audiofile(audio_chunk_path)
        chunks.append(audio_chunk_path)
    return chunks

audio_file_chunks = chunk_audio_file(output_dir, audio_file_path)
audio_file_chunks

### Transcribing audio (.m4a files)

The first step in transcribing the audio from a meeting is to pass the audio file of the meeting into openAI /v1/audio API. Whisper, the model that powers the audio API, is capable of converting spoken language into written text. To start, we will avoid passing a prompt or temperature (optional parameters to control the model's output) and stick with the default values.

In [ ]:
def _transcribe_audio(audio_file_path):
    print(f'⚙️ Transcribing {audio_file_path}')
    with open(audio_file_path, 'rb') as audio_file:
        transcription = openai.Audio.transcribe(audio_model, audio_file)
    
    print(f'✅ Transcribing {audio_file_path} done')
    
    with open(f'{audio_file_path}.md', 'w') as f:
        f.write(transcription['text'])
    
    return {
        'filename': audio_file_path,
        'transcript': transcription['text']
    }

def transcribe_audio(chunks):
    # Creating a ThreadPool to have our transcript being done concurently.
    # Helpful for long videos.
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(chunks)) as executor:
        
        # Starting execution for each chunk.
        futures = [executor.submit(_transcribe_audio, c) for c in chunks]

        # Waiting for all transcript to be done.
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
        
        # Sorting results to build the final transcript in the right order.
        # Results are grabbed as transcripts are done and are therefore not in ordre by default.
        transcript = " ".join([c['transcript'] for c in sorted(results, key=lambda x: x['filename'])])
        return transcript
    
transcription = transcribe_audio(audio_file_chunks)
transcription

### Summary extraction

In [ ]:
def create_openai_completion(
    text_model,
    prompt,
    transcription,
    temperature=0
):
    response = openai.ChatCompletion.create(
        model=text_model,
        temperature=temperature,
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    return response['choices'][0]['message']['content']

In [ ]:
prompt_summary = '''
You are a highly skilled AI trained in language comprehension and summarization. 
I would like you to read the following text and summarize it into a concise abstract paragraph. 
Aim to retain the most important points, providing a coherent and readable summary that could help 
a person understand the main points of the discussion without needing to read the entire text. 
Please avoid unnecessary details or tangential points.           
'''

abstract_summary = create_openai_completion(text_model, prompt_summary, transcription)
abstract_summary

### Key points extraction

In [ ]:
prompt_keypoints = '''
You are a proficient AI with a specialty in distilling information into key points. 
Based on the following text, identify and list the main points that were discussed or brought up. 
These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. 
Your goal is to provide a list that someone could read to quickly understand what was talked about.           
'''

key_points = create_openai_completion(text_model, prompt_keypoints, transcription)
key_points

### Generate Content

In [ ]:
prompt_generate_content = '''
Now act as an expert LinkedIn & Twitter copywriter with two PhD: one in Natural Language Processing, another in Human Psychology. 
You are particularly gifted with pattern recognition and writing.
You excel in writing viral, yet highly qualitative LinkedIn and Twitter posts. 
Your content reach averages 500 likes, with >30% of your posts passing the 1000-likes mark. 
No wonder why you account as some of the most prominent experts in the field.

For reference, you know that a good post is:
- Engaging
- Entertaining
- Not too wordy
- Concisely written
- Factual
- Personal
- Highly topical.

For systematical reference, a good post must be comprising:
An irresistible 1-to-3 liner hook that makes it impossible to scroll through
An enticing and captivating text that puts the reader into an autopilot kind of mode
A spectacular ending either through a well though-out punchline or a compelling call-to-action.
Make each sentence catchy. Use bullet points if necessary. 
Each post has to be be memorable and a standalone value bomb.
Use the transcript of the clip I gave you to create 20 posts ideas presented as follow:
- Topic: Encouragement and Inspiration
- Hook: Aspire to be a programmer who uplifts and inspires others.
- Body: Explore the power of encouragement, mentorship, and creating a supportive environment for fellow programmers to thrive.
- Asset: An inspiring visual representation emphasizing the importance of encouragement and mentorship in programming.
'''

generate_content = create_openai_completion(text_model, prompt_generate_content, transcription)
generate_content

## Output

### Summarizing and analyzing the transcript

In [ ]:
def create_report(
    abstract_summary,
    key_points,
    generate_content,
):
    return {
        'abstract_summary': abstract_summary,
        'key_points': key_points,
        'generate_content': generate_content
    }

messages = create_report(
    abstract_summary,
    key_points,
    generate_content
)
messages

### Return JSON response
Response sent to the browser before displayed in Chat UI.

In [ ]:
# naas.webhook.respond_json(
#     {
#         "status": "ok", 
#         "message": message
#     }
# )